# ETUDE DYNAMYQUE DES METROS DE PARIS

## Introduction

Nous allons faire une étude sur le nombre de personne qui attentte a chaque métros.

parametre

* nb : nb de personne
* ar : % de personne qui arrive 
* so : % de perssonne qui sort de la gare
* ta : temps d'arriver du metro
* cm : capacité du metro



parramettre plus tard
* temps pour une perssonne a rentré / sortir

### Les importation

In [ ]:
%matplotlib inline

from pylab import *

import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
#ligne;nom;niem_arret;
ar = 0
so = 0
ta = 2
cm = 0.5
#tj = compteur

#def update():
    

# initialisation
on va commencer par deux gare relier entre elles.  
hypothèse :  
 * deplacement wagon instantané

In [ ]:
N = 1000

class Station:
    def __init__(self, ar, so, ta, cm):
        self.ar = ar # % d'arrivant dans la gare
        self.so = so # % de partant dans la gare
        self.ta = ta # temps entre chaque metro
        self.cm = cm # capacité d'un metro
        self.nb = N # nombre de personne dans le metro
        self.t = 0   # initialisatien du temps de la gare
        self.next_s = self
    
    
    def update(self): # fonction qui actualise les donnés
        if self.so >= np.random.choice(100):
            n = np.random.choice(10)
            if self.nb - n >= 0:
                self.nb = self.nb - n
            else:
                self.nb = 0
        if self.ar >= np.random.choice(100):
            self.nb += np.random.choice(10)
        self.t += 1
        if self.t > ta:
            if self.nb >= self.cm:
                self.next_s.nb += self.cm
                self.nb = self.nb - self.cm
            else:
                self.next_s.nb += self.nb
                self.nb = 0
            self.t=0
                
    
    def next_station(self, station):
        self.next_s = station

In [ ]:
nb_step = 5

cm = 700

ar1 = 50
so1 = 50
ta1 = 1*60

ar2 = 50
so2 = 50
ta2 = 1*60

List_Gare = []
List_Gare.append(Station(ar1, so1, ta1, cm))
List_Gare.append(Station(ar2, so2, ta2, cm))
List_Gare[0].next_station(List_Gare[1])
List_Gare[1].next_station(List_Gare[0])

## une simulation

In [ ]:
def simulation(L):
    time = [] # liste qui represente le temps 
    nb_tot = []
    nb_station = []
    for i in range(nb_step):
        nb = 0
        time.append(i)
        for j in L:
            j.update()
        k = 0
        for j in L:
            if k >= len(nb_station):
                nb_station.append([])
            nb_station[k].append(j.nb)
            nb += j.nb
            k+=1
        nb_tot.append(nb)
    return (time, nb_tot, nb_station)


simulation(List_Gare)
    

In [ ]:
time, nb_tot, nb_station = simulation(List_Gare)
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(14,7))
axes[0].plot(time, nb_tot, 'b')
axes[0].set_xlabel("temps")
axes[0].set_ylabel("nb de personne au total")
for i in nb_station:
    axes[1].plot(time, i)
axes[1].set_xlabel("temps")
axes[1].set_ylabel("nb de personne par station")


## moyenner

In [ ]:
time, nb_tot_moy, nb_station_moy = simulation(List_Gare)

repet = 100

for j in range(len(nb_tot)):
    nb_tot_moy[j] = nb_tot_moy[j] / repet

for j in range(len(nb_station)):
    for k in range(len(nb_station[j])):
        nb_station[j][k] = nb_station[j][k] / repet
    
for i in range(repet - 1):
    time, nb_tot, nb_station =simulation(List_Gare)
    for j in range(len(nb_tot)):
        nb_tot_moy[j] += nb_tot[j] / repet
    for j in range(len(nb_station)):
        for k in range(len(nb_station[j])):
            nb_station[j][k] += nb_station[j][k] / repet

    
    
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(14,7))
axes[0].plot(time, nb_tot, 'b')
axes[0].set_xlabel("temps")
axes[0].set_ylabel("nb de personne au total")
for i in nb_station:
    axes[1].plot(time, i)
axes[1].set_xlabel("temps")
axes[1].set_ylabel("nb de personne par station")

# Test simulation Ed ligne M3b

In [1]:
%matplotlib inline
import os
import numpy as np
from LibMetroIti_NB import *

if os.name == 'nt':
    L=listeMetroCsv(".\\data\\liste stations.csv") #list([ligne,station,num_station])
    Lex=listeMetroCsvStation(".\\data\\liste stations exceptions-fourche2.csv") #list([ligne,station,num_station])
    Lf=listeMetroCsvStation(".\\data\\liste stations fourche1.csv") #list([ligne,station,num_station])
    Linfos=listeMetroCsvInfos(".\\data\\infos_metro.csv") #list([ligne,nb_arrets,temps de trajet, capacité rame])
else:
    L=listeMetroCsv("./data/liste stations.csv") #list([ligne,station,num_station])
    Lex=listeMetroCsvStation("./data/liste stations exceptions-fourche2.csv") #list([ligne,station,num_station])
    Lf=listeMetroCsvStation("./data/liste stations fourche1.csv") #list([ligne,station,num_station])
    Linfos=listeMetroCsvInfos("./data/infos_metro.csv") #list([ligne,nb_arrets,temps de trajet, capacité rame])


print(listage_stations_itineraire("PORTE DE VANVES","M13","JUSSIEU","M10"))

[['PORTE DE VANVES', 'PLAISANCE', 'M13', 210.0, '574'], ['PLAISANCE', 'PERNETY', 'M13', 210.0, '574'], ['PERNETY', 'GAITE', 'M13', 210.0, '574'], ['GAITE', 'MONTPARNASSE', 'M13', 210.0, '574'], ['MONTPARNASSE', 'SAINT-PLACIDE', 'M4', 330.0, '574'], ['SAINT-PLACIDE', 'SAINT-SULPICE', 'M4', 330.0, '574'], ['SAINT-SULPICE', 'SAINT-GERMAIN-DES-PRES', 'M4', 330.0, '574'], ['SAINT-GERMAIN-DES-PRES', 'ODEON', 'M4', 330.0, '574'], ['ODEON', 'CLUNY-LA-SORBONNE', 'M10', 300.0, '574'], ['CLUNY-LA-SORBONNE', 'MAUBERT-MUTUALITE', 'M10', 300.0, '574'], ['MAUBERT-MUTUALITE', 'CARDINAL-LEMOINE', 'M10', 300.0, '574'], ['CARDINAL-LEMOINE', 'JUSSIEU', 'M10', 300.0, '574']]


In [ ]:
L_M3b=list()
for var in L:
    if var[0]=="M3b":
        L_M3b.append(var)
print(L_M3b)   

In [ ]:
#paramètres
temps_max= 3*4*60 #3 fois * 4 mins * 60 sec
nb_utilisateurs = 10
nb_metro = 4

In [ ]:
def distrib_itineraire(nb_utilisateurs):
    base_utilisateurs=[]
    for i in range(1,nb_utilisateurs+1,1):
        x=0
        y=0
        while (x==y):
            x=dictMetroCsv(L)["M3b"][np.random.choice(len(dictMetroCsv(L)["M3b"]))]
            y=dictMetroCsv(L)["M3b"][np.random.choice(len(dictMetroCsv(L)["M3b"]))]
        base_utilisateurs.append([i,x,y,x]) #num_utilisateur, depart, arrivee, station actuelle
    return base_utilisateurs
print(distrib_itineraire(nb_utilisateurs))

In [ ]:
def initialisation():
    base_utilisateurs=distrib_itineraire(nb_utilisateurs)
    base_station_M3b = list()
    
    for station in L_M3b:
        for utilisateur in base_utilisateurs:
            if utilisateur[3]==station:
                base_station_M3b.append()

        
    return base_station_M3b
print(initialisation())

In [ ]:
def simulation():
    time = [0] # liste qui represente le temps 
    nb_utilisateur_total_M3b = []
    nb_utilisateur_station_M3b = dict()
    base_metro_M3b=dict()
    base_station_M3b, base_utilisateurs=initialisation()
    
    for station in dictMetroCsv(L)["M3b"]:
        base_metro_M3b[station]=[]
    
    for i in range(1,nb_metro+1,1) :
        if i%2!=0:
            base_metro_M3b[dictMetroCsv(L)["M3b"][0]].append(i)
        elif i%2==0:
            base_metro_M3b[dictMetroCsv(L)["M3b"][-1]].append(i)
    
    for t in range(1,temps_max,1):
        if t%180==0 :
            if base_metro_M3b[dictMetroCsv(L)["M3b"][0]]!=[] and (base_station_M3b["GAMBETTA"]!=()
            
            
    
print(simulation())



In [ ]:
print(base_utilisateurs)